In [1]:
import numpy as np
from scipy.sparse import lil_matrix as csr


class MatrixDecomposition:

    def __init__(self, a):
        self.n = len(a)
        self.a = csr(a, dtype=float)
        self.l = csr((self.n, self.n), dtype=float)
        self.u = csr(np.eye(self.n), dtype=float)
        self._create_lu()

    def _create_lu(self):
        for i in range(self.n):
            for j in range(self.n):
                s = sum([self.l[i, k] * self.u[k, j] for k in range(min(j, i))])
                if i <= j:
                    self.u[i, j] = self.a[i, j] - s
                else:
                    self.l[i, j] = (self.a[i, j] - s) / self.u[j, j]

    def solve_by_gauss(self, b):
        # finding solutions ly = b
        y = np.zeros(self.n)
        for i in range(self.n):
            s = sum([self.l[i, j] * y[j] for j in range(i)])
            y[i] = b[i] - s

        # finding solution for ux = y
        x = np.zeros(self.n)
        for i in reversed(range(self.n)):
            s = sum([self.u[i, j] * x[j] for j in range(i + 1, self.n)])
            x[i] = (y[i] - s) / self.u[i, i]

        return x

    def inverse_matrix(self):
        eye = np.eye(self.n)
        inv = np.array([self.solve_by_gauss(eye[i]) for i in range(self.n)])
        return inv.transpose()

    def solve_by_seidel(self, b, eps):
        x = np.zeros(self.n)
        converges = False
        while not converges:
            for i in range(self.n):
                s = sum([self.a[i, j] * x[j] for j in range(self.n) if i != j])
                x[i] = (b[i] - s) / self.a[i, i]
            norm = 0.0
            for i in range(self.n):
                s = sum([self.a[i, j] * x[j] for j in range(self.n)])
                norm += (s - b[i]) ** 2
            if norm <= eps ** 2:
                converges = True

        return x

In [2]:
from random import randrange
from scipy.sparse import diags


class MatrixGeneration:

    @staticmethod
    def tridiagonal(n):
        r = randrange(2, 10)
        k = [(r // 2) * np.ones(n - 1), r * np.ones(n), (r // 2) * np.ones(n - 1)]
        offset = [-1, 0, 1]
        return diags(k, offset).toarray()

    @staticmethod
    def hilbert(n):
        return np.array([[1 / (i + j + 1) for i in range(n)] for j in range(n)])

    @staticmethod
    def right(a):
        f = np.array([sum(a[i, j] * (j + 1) for j in range(len(a))) for i in range(len(a))])
        return f

In [3]:
import time
from functools import wraps


%load_ext memory_profiler

def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.time()
        result = function(*args, **kwargs)
        t1 = time.time()
        print("Total time running '%s': %s seconds" % (function.__name__, str(t1 - t0)))
        return result

    return function_timer


class Analyzer:

    @staticmethod
    def analyze_tridiagonal():
        %memit Analyzer.analyze_gauss(10, MatrixGeneration.tridiagonal)
        %memit Analyzer.analyze_seidel(10, MatrixGeneration.tridiagonal)
        %memit Analyzer.analyze_gauss(50, MatrixGeneration.tridiagonal)
        %memit Analyzer.analyze_seidel(50, MatrixGeneration.tridiagonal)
        %memit Analyzer.analyze_gauss(100, MatrixGeneration.tridiagonal)
        %memit Analyzer.analyze_seidel(100, MatrixGeneration.tridiagonal)

    @staticmethod
    def analyze_hilbert():
        %memit Analyzer.analyze_gauss(10, MatrixGeneration.hilbert)
        %memit Analyzer.analyze_seidel(10, MatrixGeneration.hilbert)
        %memit Analyzer.analyze_gauss(50, MatrixGeneration.hilbert)
        %memit Analyzer.analyze_seidel(50, MatrixGeneration.hilbert)
        %memit Analyzer.analyze_gauss(100, MatrixGeneration.hilbert)
        %memit Analyzer.analyze_seidel(100, MatrixGeneration.hilbert)

    @staticmethod
    @fn_timer
    def analyze_gauss(n, method):
        print(f'\'{method.__name__}\' {n}')
        matrix = method(n)
        right = MatrixGeneration.right(matrix)
        matrix_decomposition = MatrixDecomposition(matrix)
        gauss = matrix_decomposition.solve_by_gauss(right)

    @staticmethod
    @fn_timer
    def analyze_seidel(n, method):
        print(f'\'{method.__name__}\' {n}')
        matrix = method(n)
        right = MatrixGeneration.right(matrix)
        matrix_decomposition = MatrixDecomposition(matrix)
        seidel = matrix_decomposition.solve_by_seidel(right, 1e-3)


In [4]:
Analyzer.analyze_tridiagonal()

'tridiagonal' 10
Total time running 'analyze_gauss': 0.0045719146728515625 seconds
'tridiagonal' 10
Total time running 'analyze_gauss': 0.00423121452331543 seconds
'tridiagonal' 10
Total time running 'analyze_gauss': 0.0038299560546875 seconds
peak memory: 94.34 MiB, increment: -2.69 MiB
'tridiagonal' 10
Total time running 'analyze_seidel': 0.02301788330078125 seconds
'tridiagonal' 10
Total time running 'analyze_seidel': 0.010744810104370117 seconds
'tridiagonal' 10
Total time running 'analyze_seidel': 0.03342604637145996 seconds
peak memory: 94.66 MiB, increment: -2.19 MiB
'tridiagonal' 50
Total time running 'analyze_gauss': 0.15100336074829102 seconds
'tridiagonal' 50
Total time running 'analyze_gauss': 0.15372085571289062 seconds
peak memory: 95.27 MiB, increment: -1.69 MiB
'tridiagonal' 50
Total time running 'analyze_seidel': 4.2965991497039795 seconds
peak memory: 97.12 MiB, increment: 0.01 MiB
'tridiagonal' 100
Total time running 'analyze_gauss': 1.1336050033569336 seconds
peak m

In [5]:
Analyzer.analyze_hilbert()

'hilbert' 10
Total time running 'analyze_gauss': 0.003874063491821289 seconds
'hilbert' 10
Total time running 'analyze_gauss': 0.003064870834350586 seconds
'hilbert' 10
Total time running 'analyze_gauss': 0.003092050552368164 seconds
peak memory: 94.57 MiB, increment: -2.81 MiB
'hilbert' 10
Total time running 'analyze_seidel': 0.08898591995239258 seconds
'hilbert' 10
Total time running 'analyze_seidel': 0.09050321578979492 seconds
peak memory: 95.16 MiB, increment: -1.98 MiB
'hilbert' 50
Total time running 'analyze_gauss': 0.16630983352661133 seconds
'hilbert' 50
Total time running 'analyze_gauss': 0.15967988967895508 seconds
peak memory: 95.78 MiB, increment: -1.34 MiB
'hilbert' 50
Total time running 'analyze_seidel': 27.14517879486084 seconds
peak memory: 97.60 MiB, increment: 0.00 MiB
'hilbert' 100
Total time running 'analyze_gauss': 1.2445569038391113 seconds
peak memory: 97.52 MiB, increment: 0.00 MiB
'hilbert' 100
Total time running 'analyze_seidel': 192.78250694274902 seconds
pe